# Service

A set of tables for handling service activities


In [1]:
#| default_exp Service

In [2]:
#| export
#| hide
from biobib.table import Table, sh
from biobib.sheet import Sheet
from nbdev import show_doc
import pandas as pd

In [3]:
#| hide
from fastcore.test import *
from fastcore.test import *
from fastcore.basics import *
from fastcore.foundation import *

In [4]:
service_types = {
    'P': 'Professional',
    'U': 'University',
    'D': 'Departmental'
}

### Define a template for service activities

Here is a template we can use for service activities. It generates a longtable which can span multiple pages, and provides the year, role, and type of each service activity.


In [5]:
service_template = r'''% UC Bio-bib Professional Generic Service Table
% Created on \VAR{created}

\begin{longtable}{llp{12cm}}
Year & Role & Service
\hline 
\endfirsthead


\multicolumn{3}{c}
{ \VAR{table_name} - continued from previous page }

Year & Role & Service

\hline 
\endhead

\multicolumn{3}{c}
{ \VAR{table_name} continued on next page }
\endfoot

\hline \hline
\endlastfoot

\BLOCK{for service in items}
\VAR{service.Year} & \VAR{service.Role} & \VAR{service.Committee}
\BLOCK{endfor}
\end{longtable}

'''

In [6]:
class Service(Table):
    """ 
    A Service class of Table for use in generating biobibs and CVs 
    """
    def __init__(
            self,
            sheet:Sheet=None, # Google sheet object
            worksheet:str='Service',   # worksheet name in google sheet
            csv_file:str=None, # optional csv file (if using csv files) 
            category:str='U',  # type of service (P = professional, U = university, D = department)
            table_name:str=None, # Will be determined from category.
            cumulative:bool=False, # Is this table cumulative? 
            template=service_template # template content
    ):
        if not table_name:
            table_name = service_types[category] + ' Service'
        super().__init__(
            sheet=sheet,
            worksheet=worksheet, 
            table_name=table_name,
            csv_file=csv_file, 
            template=template)
        self.cumulative = cumulative
        self.category = category
        self.df = self.clean_df()

### Provide a data cleaning function for the service table



In [7]:
#| export

@patch
def clean_df(self:Service,
             sort_by:str='Year', # variable to sort by
             ascending:bool=True # ascending?
            )->pd.DataFrame:  # cleaned dataframe
    """
    Clean the Service table.
    
    """
    df = Table.table_clean_df(self)
    df = df.sort_values(by=[sort_by], ascending=[ascending])
    return df

In [8]:
sh.get_dataframe

<bound method Sheet.get_dataframe of <biobib.sheet.Sheet object at 0x17b2d1090>>

In [9]:
service_table = Service(sheet=sh,worksheet='Service',table_name='Test',template=service_template)

In [10]:
test_eq(service_table.table_name,'Test')
test_eq(list(service_table.df.columns),['Year', 'Type', 'Role', 'Committee', 'Notes', 'Eval', 'Sort Year'])

In [11]:
print(service_table.render_template())

% UC Bio-bib Professional Generic Service Table
% Created on 2023-06-28 18:48

\begin{longtable}{llp{12cm}}
Year & Role & Service
\hline 
\endfirsthead


\multicolumn{3}{c}
{ Test - continued from previous page }

Year & Role & Service

\hline 
\endhead

\multicolumn{3}{c}
{ Test continued on next page }
\endfoot

\hline \hline
\endlastfoot

2021 - 2022 & Chair & Dangermond Endowed Chair Search Committee, Office of Research
2021 - 2022 & Chair & NCEAS Campus Advisory Committee 
2022 - 2023 & Member & Information Technology Council, Office of the Chief Information Officer
2022 - 2023 & Member & Disaster Resilience Network Campus-based Committee
2022 - 2023 & Associate Vice Chancellor & Office of Research
2022 - 2023 & Participating Faculty & Interdepartmental PhD Emphasis in Environment and Society
\end{longtable}



In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()